<a href="https://colab.research.google.com/github/DavidB363/file-store/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# My Recommender System

import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv')
df=df.drop(df.columns[0],1)
print(df)

       -7.82   8.79  -9.66  -8.16  -7.52  ...  99.22  -5.63  99.23  99.24  99.25
0       4.08  -0.29   6.36   4.37  -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
1      99.00  99.00  99.00  99.00   9.03  ...  99.00  99.00  99.00  99.00  99.00
2      99.00   8.35  99.00  99.00   1.80  ...  99.00  99.00  99.00  99.00  99.00
3       8.50   4.61  -4.17  -5.39   1.36  ...   1.55   3.11   6.55   1.80   1.60
4      -6.17  -3.54   0.44  -8.50  -7.09  ...  -3.35   0.05  -9.08  -5.05  -3.45
...      ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24977   0.44   7.43   9.08   2.33   3.20  ...   9.03   6.55   8.69   8.79   7.43
24978   9.13  -8.16   8.59   9.08   0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24979  99.00  99.00  99.00  99.00  -7.77  ...  99.00  99.00  99.00  99.00  99.00
24980  99.00  99.00  99.00  99.00  -9.71  ...  99.00  99.00  99.00  99.00  99.00
24981   2.43   2.67  -3.98   4.27  -2.28  ...  99.00  99.00  99.00  99.00  99.00

[24982 rows x 100 columns]


In [0]:
arr = df.values
print(arr.shape)
#print(arr_full)

(24982, 100)


In [0]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810381 (1810381,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)
print(idx)

(array([ 5720, 17047, 16009, ...,  8297, 23141,  1827]), array([10, 66, 36, ..., 30, 73,  6]))


In [0]:
arr[idx[0][0], idx[1][0]]

-4.51

In [0]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
latent_item_features

array([[0.571, 0.112],
       [0.746, 0.709],
       [0.715, 0.065],
       [0.834, 0.504],
       [0.488, 0.13 ],
       [0.606, 0.853],
       [0.594, 0.727],
       [0.972, 0.778],
       [0.231, 0.562],
       [0.006, 0.345],
       [0.811, 0.008],
       [0.891, 0.62 ],
       [0.093, 0.694],
       [0.519, 0.192],
       [0.113, 0.433],
       [0.979, 0.321],
       [0.279, 0.508],
       [0.472, 0.866],
       [0.178, 0.788],
       [0.122, 0.961],
       [0.892, 0.384],
       [0.584, 0.04 ],
       [0.279, 0.191],
       [0.936, 0.364],
       [0.723, 0.789],
       [0.129, 0.959],
       [0.903, 0.971],
       [0.28 , 0.844],
       [0.007, 0.39 ],
       [0.284, 0.836],
       [0.941, 0.23 ],
       [0.511, 0.764],
       [0.914, 0.729],
       [0.703, 0.337],
       [0.648, 0.567],
       [0.529, 0.295],
       [0.689, 0.432],
       [0.399, 0.313],
       [0.083, 0.399],
       [0.209, 0.142],
       [0.756, 0.633],
       [0.898, 0.4  ],
       [0.082, 0.454],
       [0.1

In [0]:
latent_user_preferences

array([[0.194, 0.471],
       [0.789, 0.915],
       [0.66 , 0.666],
       ...,
       [0.063, 0.499],
       [0.687, 0.067],
       [0.676, 0.076]])

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
#def train(user_id, item_id, rating, alpha=0.01):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

#def sgd(iterations=300000):
def sgd(iterations=10):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 1) == 0:
            print(mse)

In [0]:
sgd(10)  # Note how the MSE decreases with the number of iterations

26.23987298133023
25.267571721936736
24.97442557538709
24.754304113946716
24.541623792135706
24.327490666553924
24.111022055502684
23.89299161462357
23.67463072326724
23.457293769822826
